### Study of the effect of national factors on home prices in the US


**Task:** Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

**Approach:** The following variables are chosen for the study:

1. Unemployment Rate
2. Employment Rate
3. Per capita GDP
4. Median Household Income
5. Construction Prices
6. CPI
7. Interest Rates
8. The number of new houses supplied
9. Working Population
10. Urban Population
11. Percentage of population above 65
12. Housing subsidies
13. Number of Households

As a proxy for home prices, the S&P **Case-Shiller Index** is used.

**Note:** Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the **Extract Transform Load (ETL)** method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.


#### Importing neccessary libraries

In [82]:
import numpy as np
import pandas as pd

#### Perform ETL

In [83]:
print(df_CS.columns)


Index(['DATE', 'CSUSHPISA', 'Year', 'Month', 'Per_Capita_GDP'], dtype='object')


In [84]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CSUSHPISA.csv")
print(df_CS.columns)

# Changing dtype of date column
df_CS["observation_date"] = pd.to_datetime(df_CS["observation_date"])
df_CS["observation_date"] = pd.to_datetime(df_CS["observation_date"])

df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Selecting data till JULY 2023
mask = df_CS["DATE"] <= "2025-05-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()


Index(['observation_date', 'CSUSHPISA'], dtype='object')
Shape of the CASE-SHILLER Index:-  (243, 4)


,DATE,CSUSHPISA,Year,Month
238,2024-11-01,326.006,2024,11
239,2024-12-01,327.680,2024,12
240,2025-01-01,329.481,2025,1
241,2025-02-01,330.384,2025,2
242,2025-03-01,329.393,2025,3


In [85]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([242], inplace = True)
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

print(df_CS.dtypes)
print(df_CS.head())


Unemployment Rate Data:-  (244, 2)
DATE         datetime64[ns]
CSUSHPISA           float64
Year                  int32
Month                 int32
dtype: object
        DATE  CSUSHPISA  Year  Month
0 2005-01-01    161.289  2005      1
1 2005-02-01    163.346  2005      2
2 2005-03-01    165.814  2005      3
3 2005-04-01    167.503  2005      4
4 2005-05-01    169.352  2005      5


In [86]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("EMPRATE.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
df_emp.drop([242], inplace = True)
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


shape of the Employment Rate Data:-  (243, 2)


In [87]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("GDP.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the Per Capita GDP Data:-  (81, 2)


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.


In [88]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([240])
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the Interest rate data:-  (240, 2)


In [89]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("construction_price_ppi.csv", names = ["DATE", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
df_cons_price_index.drop([240], inplace = True)
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the Construction Material Data:-  (244, 2)


In [90]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPIAUCSL"], skiprows = 1).drop([240])
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the Consumer Price Index:-  (243, 2)


In [91]:
# Monthly new house supply
df_house = pd.read_csv("monthly_house_supply.csv", names = ["DATE", "MSACSR"], skiprows = 1).drop([240])
df_house = df_house.rename(columns={'MSACSR': 'Houses'})
print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])



Shape of the monthly house supply data:-  (243, 2)


In [92]:
import pandas as pd

# Read the CSV and assign column names
df_oldpop = pd.read_csv("old_age_pop.csv", names=["DATE", "old_age_pop"], skiprows=1)

# Convert 'DATE' column safely
df_oldpop['DATE'] = pd.to_datetime(df_oldpop['DATE']).dt.strftime("%Y-%m-%d")

# Check result
print("Shape of the population data age above 65:", df_oldpop.shape)
print(df_oldpop.head())
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])



Shape of the population data age above 65: (19, 2)
         DATE  old_age_pop
0  2005-01-01    12.130728
1  2006-01-01    12.184000
2  2007-01-01    12.310108
3  2008-01-01    12.486661
4  2009-01-01    12.671093


In [93]:
# Urban Population Percent

df_urban = pd.read_csv("urbenpop2.csv")
#df_urban['DATE'] = pd.to_datetime(df_urban['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the urban population percent data:-  (19, 2)


In [94]:
# Housing Subsidies

df_subsidy = pd.read_csv("housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])



Shape of the housing subsidies:-  (19, 2)


In [95]:
# Working age population

df_working = pd.read_csv("working_age_population.csv", names = ["DATE", "LFWA64TTUSM647S"], skiprows = 1).drop([240])
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the working age population:-  (243, 2)


In [96]:
# Real Median Household Income

df_income = pd.read_csv("median_household_income.csv", names = ["DATE", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the median household income data:-  (19, 2)


In [97]:
# Total number of households

df_households = pd.read_csv("household.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])


Shape of the total households data:-  (20, 2)


In [98]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()
# Assuming the DataFrame is named df_CS

# Step 1: Rename the column
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Step 2: Convert DATE column to datetime type
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])



In [99]:
# Rename date columns to 'DATE' to standardize
df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)
df_working.rename(columns={"observation_date": "DATE"}, inplace=True)
# Repeat for others as needed...


In [100]:
# Concatenate DataFrames with monthly data into one DataFrame
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]

for i in range(len(df_bymonth)):
    if "DATE" in df_bymonth[i].columns:
        df_bymonth[i]["DATE"] = pd.to_datetime(df_bymonth[i]["DATE"])
        df_bymonth[i].set_index("DATE", inplace=True)
        df = pd.concat([df, df_bymonth[i]], axis=1)
   

print(df.shape)
df.head()


(245, 8)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Cons_Material
DATE,,,,,,,,
2005-01-01,161.289,2005.0,1.0,12767.286,190032500.0,4.4,191.6,168.6
2005-02-01,163.346,2005.0,2.0,NaN,190203400.0,4.3,192.4,170.4
2005-03-01,165.814,2005.0,3.0,NaN,190426700.0,4.1,193.1,170.0
2005-04-01,167.503,2005.0,4.0,12922.656,190492600.0,4.3,193.7,170.0
2005-05-01,169.352,2005.0,5.0,NaN,190680900.0,4.2,193.6,168.1


In [101]:
# Ensure DATE column is available in df_CS
if "DATE" not in df_CS.columns:
    if df_CS.index.name == "DATE":
        df_CS.reset_index(inplace=True)
    elif "observation_date" in df_CS.columns:
        df_CS.rename(columns={"observation_date": "DATE"}, inplace=True)

# Create 'Year' column in df_CS
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year

# Set df as base
df = df_CS.copy()

# Set index later after merging
df.set_index("DATE", inplace=False)

# List of yearly data DataFrames
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]

# Merge each with base df on 'Year'
for df1 in others:
    if "DATE" in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
    elif df1.index.name == "DATE":
        df1 = df1.reset_index()
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year

    df = pd.merge(df, df1.drop(columns="DATE"), how="left", on="Year")  # avoid duplicate DATE columns

# Finalize index
df.set_index("DATE", inplace=True)

# Done
df.head()


,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,
2005-01-01,161.289,2005,1,12767.286,NaN,113343.0,69310.0,27.651,12.130728
2005-02-01,163.346,2005,2,NaN,NaN,113343.0,69310.0,27.651,12.130728
2005-03-01,165.814,2005,3,NaN,NaN,113343.0,69310.0,27.651,12.130728
2005-04-01,167.503,2005,4,12922.656,NaN,113343.0,69310.0,27.651,12.130728
2005-05-01,169.352,2005,5,NaN,NaN,113343.0,69310.0,27.651,12.130728


In [102]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us_x,Num_Households_x,median_income_x,Subsidy_x,old_age_pop_x,urban_pop_us_y,Num_Households_y,median_income_y,Subsidy_y,old_age_pop_y
DATE,,,,,,,,,,,,,,
2005-01-01,161.289,2005,1,12767.286,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-02-01,163.346,2005,2,NaN,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-03-01,165.814,2005,3,NaN,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-04-01,167.503,2005,4,12922.656,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-05-01,169.352,2005,5,NaN,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728


In [103]:
print(df.shape)

(243, 14)


Check missing values (NAN)

In [104]:
df.isna().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP      162
urban_pop_us_x       27
Num_Households_x      3
median_income_x      15
Subsidy_x            15
old_age_pop_x        15
urban_pop_us_y       27
Num_Households_y      3
median_income_y      15
Subsidy_y            15
old_age_pop_y        15
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2002 to 2022.

**Interpolation:**

Interpolation is a mathematical technique used to estimate values that are missing in a dataset based on the values of neighboring data points. It calculates intermediate values based on the existing data.



In [105]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [106]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us_x,Num_Households_x,median_income_x,Subsidy_x,old_age_pop_x,urban_pop_us_y,Num_Households_y,median_income_y,Subsidy_y,old_age_pop_y
DATE,,,,,,,,,,,,,,
2005-01-01,161.289,2005,1,12767.286000,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-02-01,163.346,2005,2,12819.076000,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-03-01,165.814,2005,3,12870.866000,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-04-01,167.503,2005,4,12922.656000,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
2005-05-01,169.352,2005,5,12995.984667,NaN,113343.0,69310.0,27.651,12.130728,NaN,113343.0,69310.0,27.651,12.130728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-01,326.006,2024,11,29808.122000,NaN,132216.0,NaN,NaN,NaN,NaN,132216.0,NaN,NaN,NaN
2024-12-01,327.680,2024,12,29892.380000,NaN,132216.0,NaN,NaN,NaN,NaN,132216.0,NaN,NaN,NaN
2025-01-01,329.481,2025,1,29976.638000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df.dropna(inplace = True)

In [108]:
df.isna().sum()

CSUSHPISA           0
Year                0
Month               0
Per_Capita_GDP      0
urban_pop_us_x      0
Num_Households_x    0
median_income_x     0
Subsidy_x           0
old_age_pop_x       0
urban_pop_us_y      0
Num_Households_y    0
median_income_y     0
Subsidy_y           0
old_age_pop_y       0
dtype: int64

In [109]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us_x,Num_Households_x,median_income_x,Subsidy_x,old_age_pop_x,urban_pop_us_y,Num_Households_y,median_income_y,Subsidy_y,old_age_pop_y
DATE,,,,,,,,,,,,,,
2006-01-01,182.321,2006,1,13599.160000,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-02-01,183.288,2006,2,13650.581333,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-03-01,184.365,2006,3,13702.002667,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-04-01,184.329,2006,4,13753.424000,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-05-01,184.155,2006,5,13792.345333,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,309.045,2023,8,28077.453667,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819
2023-09-01,311.176,2023,9,28187.210333,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819
2023-10-01,313.262,2023,10,28296.967000,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819


In [110]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (216, 14)


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".


In [111]:
df.to_csv("prepared_dataset.csv")

In [ ]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us_x,Num_Households_x,median_income_x,Subsidy_x,old_age_pop_x,urban_pop_us_y,Num_Households_y,median_income_y,Subsidy_y,old_age_pop_y
DATE,,,,,,,,,,,,,,
2006-01-01,182.321,2006,1,13599.160000,80.099,114384.0,70080.0,28.604,12.184,80.099,114384.0,70080.0,28.604,12.184
2006-02-01,183.288,2006,2,13650.581333,80.099,114384.0,70080.0,28.604,12.184,80.099,114384.0,70080.0,28.604,12.184
2006-03-01,184.365,2006,3,13702.002667,80.099,114384.0,70080.0,28.604,12.184,80.099,114384.0,70080.0,28.604,12.184
2006-04-01,184.329,2006,4,13753.424000,80.099,114384.0,70080.0,28.604,12.184,80.099,114384.0,70080.0,28.604,12.184
2006-05-01,184.155,2006,5,13792.345333,80.099,114384.0,70080.0,28.604,12.184,80.099,114384.0,70080.0,28.604,12.184


In [113]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,urban_pop_us_x,Num_Households_x,median_income_x,Subsidy_x,old_age_pop_x,urban_pop_us_y,Num_Households_y,median_income_y,Subsidy_y,old_age_pop_y
DATE,,,,,,,,,,,,,,
2006-01-01,182.321,2006,1,13599.160000,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-02-01,183.288,2006,2,13650.581333,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-03-01,184.365,2006,3,13702.002667,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-04-01,184.329,2006,4,13753.424000,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
2006-05-01,184.155,2006,5,13792.345333,80.099,114384.0,70080.0,28.604,12.184000,80.099,114384.0,70080.0,28.604,12.184000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,309.045,2023,8,28077.453667,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819
2023-09-01,311.176,2023,9,28187.210333,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819
2023-10-01,313.262,2023,10,28296.967000,83.298,131434.0,80610.0,53.573,17.431819,83.298,131434.0,80610.0,53.573,17.431819


## To be continued...........